# 🔍 常用相关性分析方法总结
本笔记介绍了数据科学和特征工程中常用的相关性分析方法，包括适用条件、使用场景与 Python 示例。

## 1️⃣ Pearson 皮尔逊相关系数（连续 vs 连续）
- 用于衡量两个连续变量之间的线性关系
- 适合近似正态分布的变量
```python
from scipy.stats import pearsonr
r, _ = pearsonr(x, y)
```

## 2️⃣ Spearman 秩相关（非线性连续）
- 使用排名代替原始数值，适合单调但非线性关系
- 更鲁棒于异常值
```python
from scipy.stats import spearmanr
r, _ = spearmanr(x, y)
```

## 3️⃣ Kendall Tau（小样本秩相关）
- 更稳健于小样本秩相关分析
```python
from scipy.stats import kendalltau
r, _ = kendalltau(x, y)
```

## 4️⃣ Point-Biserial 相关（连续 vs 二分类）
- 连续变量与二分类变量之间的线性关系
```python
from scipy.stats import pointbiserialr
r, _ = pointbiserialr(binary_y, x)
```

## 5️⃣ Cramér’s V（分类 vs 分类）
- 用于两个分类变量之间的相关性（卡方检验基础）
```python
import scipy.stats as stats
import numpy as np

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))
```

## 6️⃣ Mutual Information（任意类型）
- 衡量变量之间的信息共享程度
- 适合非线性、分类、混合型变量
```python
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
mi_score = mutual_info_classif(X, y)  # 分类
mi_score = mutual_info_regression(X, y)  # 回归
```

## ✅ 方法选择建议表：
| 特征类型 | 目标类型 | 推荐方法 |
|-----------|------------|----------------------|
| 连续 vs 连续 | 连续 | Pearson / Spearman |
| 连续 vs 二分类 | int(0/1) | Point-Biserial |
| 分类 vs 二分类 | 分类 vs int | Mutual Information (Classif) |
| 连续 vs 分类 | 连续 vs 分类 | Mutual Information (Reg) |
| 分类 vs 分类 | 分类 | Cramér’s V |

## 🤖 自动相关性检测器（Auto Correlation Detector）
本部分介绍如何构建一个自动相关性检测器，根据变量类型动态选择合适的相关性评估方法。

### 📌 实现思路：
1. 判断变量的数据类型（数值、分类、二分类）
2. 根据变量组合自动选择方法：
   - 数值 vs 数值 → Pearson(or Spearman)
   - 数值 vs 二分类 → Point-Biserial
   - 分类 vs 分类 → Cramér’s V
   - 数值 vs 分类 → Mutual Information Regression
   - 分类 vs 二分类 → Mutual Information Classification
3. 计算相关性并输出评分表

### 🧠 实现代码（简化版）

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, pointbiserialr, chi2_contingency
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from pandas.api.types import is_numeric_dtype, is_categorical_dtype

def auto_correlation_detector(df, target):
    results = []
    for col in df.columns:
        if col == target:
            continue

        x = df[col]
        y = df[target]

        # 类型标准化
        if is_categorical_dtype(y): y = y.cat.codes
        if is_categorical_dtype(x): x = x.cat.codes

        if is_numeric_dtype(x) and is_numeric_dtype(y):
            method = 'pearson'
            score = pearsonr(x, y)[0]
        elif is_numeric_dtype(x) and set(y.unique()) <= {0, 1}:
            method = 'point-biserial'
            score = pointbiserialr(y, x)[0]
        elif not is_numeric_dtype(x) and not is_numeric_dtype(y):
            confusion_matrix = pd.crosstab(x, y)
            chi2 = chi2_contingency(confusion_matrix)[0]
            n = confusion_matrix.sum().sum()
            phi2 = chi2 / n
            r, k = confusion_matrix.shape
            method = 'cramér's V'
            score = np.sqrt(phi2 / min(k - 1, r - 1))
        elif not is_numeric_dtype(x) and set(y.unique()) <= {0, 1}:
            method = 'mutual_info_classif'
            score = mutual_info_classif(x.values.reshape(-1,1), y)[0]
        elif is_numeric_dtype(x) and not is_numeric_dtype(y):
            method = 'mutual_info_regression'
            score = mutual_info_regression(x.values.reshape(-1,1), y)[0]
        else:
            method = 'unknown'
            score = np.nan

        results.append({'Feature': col, 'Method': method, 'Score': score})

    return pd.DataFrame(results).sort_values(by='Score', ascending=False)

### ✅ 使用方式

In [ ]:
# 假设 df 是包含混合类型变量的 DataFrame，'target' 是目标列
result_df = auto_correlation_detector(df, target='target')
print(result_df)